In [1]:
from os import listdir
from os.path import isfile, join
import os.path
import openpyxl
import pandas as pd
import math
import openpyxl
import pickle
import time

## SABI 1

In [2]:
counter_file_name = 'file_processed_counter'

path1 = '/home/ec2-user/proyecto/data/datasets/SABI2/'
onlyfiles1 = [f for f in listdir(path1) if isfile(join(path1, f))]

#Cargamos en un vector, la lista de los ficheros que se van procesando, por si se cae el jupyter, no volver a procesar todos
#Sino proseguir con aquellos que no se han procesado.
if (not(os.path.isfile(counter_file_name))):
    with open(counter_file_name, 'wb') as dumpPickleCounter:
        pickle.dump(onlyfiles1, dumpPickleCounter)
else:
    with open(counter_file_name, 'rb') as dumpPickleCounter:
        onlyfiles1 = pickle.load(dumpPickleCounter)

print onlyfiles1

['J. 360001-405000.xlsx', 'G. 225001-270000.xlsx', 'D. 135001-180000.xlsx', 'F. 185001-225000.xlsx', 'C. 90001-135000.xlsx', 'K. 405001-450000.xlsx', 'L. 450001-495000.xlsx', 'H. 270001-315000.xlsx', 'A. 1-45000.xlsx', 'B. 45001-90000.xlsx', 'I. 315001-360000.xlsx', 'M. 495001-532592.xlsx', 'E. 180001-185000.xlsx']


Definimos las columnas que vamos a coger para cada DataFrame.

In [4]:
dfwork_cols = [1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,56,57,58,59,60,61,62,63,64,65,66,54,55,67,68,69,70]
dfaudit_cols = [1,2,35,36,37,38,39]
dfParticipadas_cols = [1,2,48,49,50,51,52,53]

In [16]:
df = pd.DataFrame()
dfwork = pd.DataFrame()
dfaudit = pd.DataFrame()
dfparticipadas = pd.DataFrame()

counter = 0

for f in list(onlyfiles1):
    io = join(path1, f)
    if counter==0:
        print 'Primer fichero:', f
        df = pd.read_excel(io, sheetname='Resultados', index_col=None, header=0, skiprows=0, na_values=None, thousands=None)
        
        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df.reset_index(inplace=True)
        df = df[df.Nombre != 'NAME']
        
        #DataFrames
        dfwork = df[dfwork_cols]
        dfaudit = df[dfaudit_cols]
        dfparticipadas = df[dfParticipadas_cols]

        counter = 1
    else:
        print 'Resto ficheros:', f
        df_temp = pd.read_excel(io, sheetname='Resultados',index_col=None, header=0, skiprows=0, na_values=None, thousands=None)

        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df_temp.reset_index(inplace=True)
        df_temp = df_temp[df_temp.Nombre != 'NAME']

        #DataFrames
        dfwork = dfwork.append(df_temp[dfwork_cols])
        dfaudit = dfaudit.append(df_temp[dfaudit_cols])
        dfparticipadas = dfparticipadas.append(df_temp[dfParticipadas_cols])

Primer fichero: 13 Lista1 Empresas 68401-74101.xlsx
Resto ficheros: 26 Lista1 Empresas 142502-148201.xlsx
Resto ficheros: 38 Lista1 Empresas 210902_216601.xlsx
Resto ficheros: 01 Lista1 Empresas 1-5700.xlsx
Resto ficheros: 02 Lista1 Empresas 5701-11400.xlsx
Resto ficheros: 03 Lista1 Empresas 11401 -17001.xlsx
Resto ficheros: 04 Lista1 Empresas 17402-22801.xlsx
Resto ficheros: 05 Lista1 Empresas 22802-28501.xlsx
Resto ficheros: 06 Lista1 Empresas 28502-34201.xlsx
Resto ficheros: 07 Lista1 Empresas 34202-39901.xlsx
Resto ficheros: 08 Lista1 Empresas 39902-45601.xlsx
Resto ficheros: 09 Lista1 Empresas 45602-51301.xlsx
Resto ficheros: 10 Lista1 Empresas 51302-57001.xlsx
Resto ficheros: 11 Lista1 Empresas 57002-62701.xlsx
Resto ficheros: 12 Lista1 Empresas 62702-68401.xlsx
Resto ficheros: 14 Lista1 Empresas 74102-79801.xlsx
Resto ficheros: 15 Lista1 Empresas 79802-85501.xlsx
Resto ficheros: 16 Lista1 Empresas 85502-91201.xlsx
Resto ficheros: 17 Lista1 Empresas 91201-96901.xlsx
Resto fichero

Formateamos los diferentes DataFrames que vamos a utilizar

In [31]:
cols_index = [u'Nombre', u'Código NIF']

#DataFrame de la empresa
dfwork2 = dfwork.dropna(subset=cols_index)

#DataFrame de auditores
dfaudit2 = dfaudit.dropna(subset=[u'Nombre auditor',u'Salvedades'], how='all')
dfaudit2[cols_index] = dfaudit2[cols_index].fillna(method='ffill')

#DataFrame de participadas
dfparticipadas2 = dfparticipadas.dropna(subset=[u'Participada - Nombre'])
dfparticipadas2[cols_index] = dfparticipadas2[cols_index].fillna(method='ffill')

Pasamos los DataFrames a ficheros

In [ ]:
dfwork_file = 'dfwork'
dfaudit_file = 'dfaudit'
dfparticipadas_file = 'dfparticipadas'

if (not(os.path.isfile(dfwork_file))):
    with open(dfwork_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfwork2, dumpPickleCounter)
else:
    with open(dfwork_file, 'rb') as dumpPickleCounter:
        dfwork2 = pickle.load(dumpPickleCounter)
        
if (not(os.path.isfile(dfaudit_file))):
    with open(dfaudit_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfaudit2, dumpPickleCounter)
else:
    with open(dfaudit_file, 'rb') as dumpPickleCounter:
        dfaudit2 = pickle.load(dumpPickleCounter)
        
if (not(os.path.isfile(dfparticipadas_file))):
    with open(dfparticipadas_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfparticipadas2, dumpPickleCounter)
else:
    with open(dfparticipadas_file, 'rb') as dumpPickleCounter:
        dfparticipadas2 = pickle.load(dumpPickleCounter)

## SABI 2

In [42]:
counter_file_name2 = 'file_processed_counter2'

path2 = 'SABI Lista2/'
onlyfiles2 = [f for f in listdir(path2) if isfile(join(path2, f))]

#Cargamos en un vector, la lista de los ficheros que se van procesando, por si se cae el jupyter, no volver a procesar todos
#Sino proseguir con aquellos que no se han procesado.
if (not(os.path.isfile(counter_file_name2))):
    with open(counter_file_name, 'wb') as dumpPickleCounter:
        pickle.dump(onlyfiles2, dumpPickleCounter)
else:
    with open(counter_file_name2, 'rb') as dumpPickleCounter:
        onlyfiles2 = pickle.load(dumpPickleCounter)

print onlyfiles2

['12 Lista2 Empresas 90001_97000.xlsx', '25 Lista2 Empresas 181001-188000.xlsx', '01 Lista2 Empresas 1_7500.xlsx', '02 Lista2 Empresas 7501_15000.xlsx', '03 Lista2 Empresas 15001_22501.xlsx', '04 Lista2 Empresas 22502_30001.xlsx', '05 Lista 2 Empresas 30002_37501.xlsx', '06 Lista2 Empresas 37502_45001.xlsx', '07 Lista2 Empresas 45002-52501.xlsx', '08 Lista2 Empresas 52502_60001.xlsx', '09 Lista2 Empresas 60002_67501.xlsx', '10 Lista2 Empresas 67501_75001.xlsx', '11 Lista2 Empresas 75002_82501.xlsx', '12 Lista2 Empresas 82501_90001.xlsx', '13 Lista2 Empresas 97001_104000.xlsx', '14 Lista2 Empresas 104001_111000.xlsx', '15 Lista2 Empresas 111001_118000.xlsx', '16 Lista2 Empresas 181001_125000.xlsx', '17 Lista2 Empresas 125001_132000.xlsx', '18 Lista2 Empresas 132000-139000.xlsx', '19 Lista2 Empresas 139001-146000.xlsx', '20 Lista2 Empresas 146001_153000.xlsx', '21 Lista2 Empresas 153001-160000.xlsx', '22 Lista2 Empresas 160001-167000.xlsx', '23 Lista2 Empresas 167001-174000.xlsx', '24 Li

In [48]:
dfjunta_cols = [1,2,16,17,18,19,20,21,3,4,5,6,7,8,9,10,11,12,13,14,15,22,24]
dfaccionistas_cols = [1,2,25,26,27,28,29,30,31,32]

In [49]:
df = pd.DataFrame()
dfjunta = pd.DataFrame()
dfaccionistas = pd.DataFrame()

counter = 0

for f in list(onlyfiles2):
    io = join(path2, f)
    if counter==0:
        print 'Primer fichero:', f
        df = pd.read_excel(io, sheetname='Resultados', index_col=None, header=0, skiprows=0, na_values=None, thousands=None)
        
        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df.reset_index(inplace=True)
        df = df[df.Nombre != 'NAME']
        
        #DataFrames
        dfjunta = df[dfjunta_cols]
        dfaccionistas = df[dfaccionistas_cols]

        counter = 1
    else:
        print 'Resto ficheros:', f
        df_temp = pd.read_excel(io, sheetname='Resultados',index_col=None, header=0, skiprows=0, na_values=None, thousands=None)

        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df_temp.reset_index(inplace=True)
        df_temp = df_temp[df_temp.Nombre != 'NAME']

        #DataFrames
        dfjunta = dfjunta.append(df_temp[dfjunta_cols])
        dfaccionistas = dfaccionistas.append(df_temp[dfaccionistas_cols])

Primer fichero: 12 Lista2 Empresas 90001_97000.xlsx
Resto ficheros: 25 Lista2 Empresas 181001-188000.xlsx
Resto ficheros: 01 Lista2 Empresas 1_7500.xlsx
Resto ficheros: 02 Lista2 Empresas 7501_15000.xlsx
Resto ficheros: 03 Lista2 Empresas 15001_22501.xlsx
Resto ficheros: 04 Lista2 Empresas 22502_30001.xlsx
Resto ficheros: 05 Lista 2 Empresas 30002_37501.xlsx
Resto ficheros: 06 Lista2 Empresas 37502_45001.xlsx
Resto ficheros: 07 Lista2 Empresas 45002-52501.xlsx
Resto ficheros: 08 Lista2 Empresas 52502_60001.xlsx
Resto ficheros: 09 Lista2 Empresas 60002_67501.xlsx
Resto ficheros: 10 Lista2 Empresas 67501_75001.xlsx
Resto ficheros: 11 Lista2 Empresas 75002_82501.xlsx
Resto ficheros: 12 Lista2 Empresas 82501_90001.xlsx
Resto ficheros: 13 Lista2 Empresas 97001_104000.xlsx
Resto ficheros: 14 Lista2 Empresas 104001_111000.xlsx
Resto ficheros: 15 Lista2 Empresas 111001_118000.xlsx
Resto ficheros: 16 Lista2 Empresas 181001_125000.xlsx
Resto ficheros: 17 Lista2 Empresas 125001_132000.xlsx
Resto 

In [50]:
#DataFrame de la junta
dfjunta2 = dfjunta.dropna(subset=cols_index)

#DataFrame de accionistas
dfaccionistas2 = dfaccionistas.dropna(subset=[u'Nombre accionista'])
dfaccionistas2[cols_index] = dfaccionistas2[cols_index].fillna(method='ffill')

Pasamos los DataFrames a ficheros

In [ ]:
dfjunta_file = 'dfjunta'
dfaccionistas_file = 'dfaccionistas'

if (not(os.path.isfile(dfjunta_file))):
    with open(dfjunta_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfwork2, dumpPickleCounter)
else:
    with open(dfjunta_file, 'rb') as dumpPickleCounter:
        dfjunta2 = pickle.load(dumpPickleCounter)
        
if (not(os.path.isfile(dfaccionistas_file))):
    with open(dfaccionistas_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfaccionistas2, dumpPickleCounter)
else:
    with open(dfaccionistas_file, 'rb') as dumpPickleCounter:
        dfaccionistas2 = pickle.load(dumpPickleCounter)


## SABI 3

In [2]:
counter_file_name = 'file_processed_counter'

path3 = '../../data/datasets/SABI2'
onlyfiles3 = [f for f in listdir(path3) if isfile(join(path3, f))]

#Cargamos en un vector, la lista de los ficheros que se van procesando, por si se cae el jupyter, no volver a procesar todos
#Sino proseguir con aquellos que no se han procesado.
if (not(os.path.isfile(counter_file_name))):
    with open(counter_file_name, 'wb') as dumpPickleCounter:
        pickle.dump(onlyfiles3, dumpPickleCounter)
else:
    with open(counter_file_name, 'rb') as dumpPickleCounter:
        onlyfiles3 = pickle.load(dumpPickleCounter)

print onlyfiles3

['J. 360001-405000.xlsx', 'G. 225001-270000.xlsx', 'D. 135001-180000.xlsx', 'F. 185001-225000.xlsx', 'C. 90001-135000.xlsx', 'K. 405001-450000.xlsx', 'L. 450001-495000.xlsx', 'H. 270001-315000.xlsx', 'A. 1-45000.xlsx', 'B. 45001-90000.xlsx', 'I. 315001-360000.xlsx', 'M. 495001-532592.xlsx', 'E. 180001-185000.xlsx']


Definimos las columnas que vamos a coger para cada DataFrame.

In [3]:
dfwork_cols = [1,2,3,4,5,7,8,11,20,21]
dfaudit_cols = [1,2,3,16,17]

In [4]:
df = pd.DataFrame()
dfwork = pd.DataFrame()
dfaudit = pd.DataFrame()

counter = 0

for f in list(onlyfiles3):
    io = join(path3, f)
    if counter==0:
        print 'Primer fichero:', f
        df = pd.read_excel(io, sheetname='Resultados', index_col=None, header=0, skiprows=0, na_values=None, thousands=None)
        
        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df.reset_index(inplace=True)
        df = df[df.Nombre != 'NAME']
        
        #DataFrames
        dfwork = df[dfwork_cols]
        dfaudit = df[dfaudit_cols]

        counter = 1
    else:
        print 'Resto ficheros:', f
        df_temp = pd.read_excel(io, sheetname='Resultados',index_col=None, header=0, skiprows=0, na_values=None, thousands=None)

        # Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
        df_temp.reset_index(inplace=True)
        df_temp = df_temp[df_temp.Nombre != 'NAME']

        #DataFrames
        dfwork = dfwork.append(df_temp[dfwork_cols])
        dfaudit = dfaudit.append(df_temp[dfaudit_cols])

Primer fichero: J. 360001-405000.xlsx
Resto ficheros: G. 225001-270000.xlsx
Resto ficheros: D. 135001-180000.xlsx
Resto ficheros: F. 185001-225000.xlsx
Resto ficheros: C. 90001-135000.xlsx
Resto ficheros: K. 405001-450000.xlsx
Resto ficheros: L. 450001-495000.xlsx
Resto ficheros: H. 270001-315000.xlsx
Resto ficheros: A. 1-45000.xlsx
Resto ficheros: B. 45001-90000.xlsx
Resto ficheros: I. 315001-360000.xlsx
Resto ficheros: M. 495001-532592.xlsx
Resto ficheros: E. 180001-185000.xlsx


Formateamos los diferentes DataFrames que vamos a utilizar

In [6]:
dfwork.head(2)

,Nombre,Código NIF,Número BvD,Fecha constitución,Estado,Código primario CNAE 2009,Incidencias Judiciales,Reclamaciones administrativas,Coordenada - X,Coordenada - Y
1,ESPACIO ELECTRICO SL,B84860568,ESB84860568,2006-09-25 00:00:00,Activa,4321,NaN,NaN,-3.67824,40.350476
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
dfaudit.head(2)

,Nombre,Código NIF,Número BvD,Nombre auditor,Salvedades
1,ESPACIO ELECTRICO SL,B84860568,ESB84860568,NaN,NaN
2,ESPACIO ELECTRICO SL,B84860568,ESB84860568,NaN,NaN


In [15]:
cols_index = [u'Nombre', u'Código NIF', u'Número BvD']

#DataFrame de la empresa
#Quitar bancos (CNAE 641)
dfwork3 = dfwork.dropna(subset=cols_index)

#DataFrame de auditores
dfaudit[cols_index] = dfaudit[cols_index].fillna(method='ffill')
dfaudit3 = dfaudit.dropna(subset=[u'Nombre auditor',u'Salvedades'], how='all')


Pasamos los DataFrames a ficheros

In [17]:
dfwork_file = 'dfwork3'
dfaudit_file = 'dfaudit3'

if (not(os.path.isfile(dfwork_file))):
    with open(dfwork_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfwork3, dumpPickleCounter)
else:
    with open(dfwork_file, 'rb') as dumpPickleCounter:
        dfwork3 = pickle.load(dumpPickleCounter)
        
if (not(os.path.isfile(dfaudit_file))):
    with open(dfaudit_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfaudit3, dumpPickleCounter)
else:
    with open(dfaudit_file, 'rb') as dumpPickleCounter:
        dfaudit3 = pickle.load(dumpPickleCounter)


In [18]:
dfaudit3[dfaudit3[u'Nombre auditor'] == 'MGR DE AUDITORIA Y CONSULTORIA S.L.']

,Nombre,Código NIF,Número BvD,Nombre auditor,Salvedades
4426,ALVAREZ FRA SA,A28995231,ESA28995231,MGR DE AUDITORIA Y CONSULTORIA S.L.,Aprobado
4428,ALVAREZ FRA SA,A28995231,ESA28995231,MGR DE AUDITORIA Y CONSULTORIA S.L.,Aprobado
58866,RAILTECH SUFETRA SA,A08008567,ESA08008567,MGR DE AUDITORIA Y CONSULTORIA S.L.,Aprobado
58867,RAILTECH SUFETRA SA,A08008567,ESA08008567,MGR DE AUDITORIA Y CONSULTORIA S.L.,Aprobado
58869,RAILTECH SUFETRA SA,A08008567,ESA08008567,MGR DE AUDITORIA Y CONSULTORIA S.L.,Aprobado


## SABI 4 (CNAE 641)

Definimos las columnas que vamos a coger para cada DataFrame.

In [2]:
dfbanks_cols = [1,2,3,7]

In [5]:
df = pd.DataFrame()
dfbanks = pd.DataFrame()

f = '/home/ec2-user/proyecto/tfm/notebooks/CNAE 641.xlsx'

print 'Primer fichero:', f

df = pd.read_excel(f, sheetname='Resultados', index_col=None, header=0, skiprows=0, na_values=None, thousands=None)
        
# Eliminamos las filas que hacen de cabecera en los ficheros originales y reseteamos el índice
df.reset_index(inplace=True)
df = df[df.Nombre != 'NAME']
        
#DataFrames
dfbanks = df[dfbanks_cols]

Primer fichero: /home/ec2-user/proyecto/tfm/notebooks/CNAE 641.xlsx


Formateamos los diferentes DataFrames que vamos a utilizar

In [7]:
cols_index = [u'Nombre', u'Código NIF', u'Número BvD']

#DataFrame de la empresa
dfbanks = dfbanks.dropna(subset=cols_index)

Pasamos los DataFrames a ficheros

In [9]:
dfbanks_file = 'dfbanks3'

if (not(os.path.isfile(dfbanks_file))):
    with open(dfbanks_file, 'wb') as dumpPickleCounter:
        pickle.dump(dfbanks, dumpPickleCounter)
else:
    with open(dfbanks_file, 'rb') as dumpPickleCounter:
        dfbanks = pickle.load(dumpPickleCounter)